In [24]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import json
import os
from dotenv import load_dotenv, dotenv_value
llm = ChatOpenAI(temperature=0,model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"))

load_dotenv()

In [26]:
### Load Document ###
# Update to handle PDF document loading
pdf_path = "Dental_Assitant_Book.pdf"  # Replace with your PDF file path
loader = PyPDFLoader(pdf_path,extract_images=True)
docs = loader.load()

### Split Document ###
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

C:\Users\admin\anaconda3\envs\GenAIWorkSpace\lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:125: UserWarning: Unknown PDF Filter!
  warnings.warn("Unknown PDF Filter!")


In [27]:
### Construct Retriever ###
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")))
retriever = vectorstore.as_retriever()

In [44]:
### Contextualize Question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer Question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise. Make sure to have proper formatted structure

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Statefully Manage Chat History ###
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [51]:
response = conversational_rag_chain.invoke(
        {"input": "Advantages and Disadvantages of Digital Radiography "},
        config={
            "configurable": {"session_id": "456"}
        },
    )["answer"]
print(response, end='\n')

**Advantages:**
1. Immediate viewing of images, allowing for quicker clinical decisions.
2. Requires significantly less radiation compared to conventional film-based radiography.
3. Eliminates the need for chemical processing and allows for easy electronic transfer of images.

**Disadvantages:**
1. High initial setup costs for digital radiography systems.
2. Sensors can be fragile and expensive to replace if damaged.
3. Users may require time to learn and become comfortable with the software.


In [70]:
retriever.invoke('Advantages and Disadvantages of Digital Radiography ')

[Document(id='d40ef31b-1e9b-4c9f-b9fa-c1e01a9415b0', metadata={'page': 1740, 'source': 'riya dental assisting book.pdf'}, page_content='BOX 39.1\n\u2003\nAdvantages and Disadvantages of Digital\nRadiography\nAdvantages\nImmediate Viewing of Images\nIn conventional radiography, reading the image is delayed because the film\nmust be processed. Being able to immediately view the image is a significant\nclinical advantage\nNote:\n Only charge-coupled devices (CCDs) provide immediate viewing.\nPhosphorous plate digital technology requires that the irradiated sensor be\nplaced in a processing device to scan it and put the information into a\ncomputer, so that the image can be viewed.\nLess Radiation\nDigital radiography requires \nmuch less x-radiation\n than conventional film-\nbased radiography because the sensor is \nmore sensitive\n to x-rays than is\nconventional film. Exposure times for digital radiography are 70 to 80\npercent less than those required for radiography with conventional

In [52]:
response = conversational_rag_chain.invoke(
        {"input": "Expand on point number 2"},
        config={
            "configurable": {"session_id": "456"}
        },
    )["answer"]
print(response, end='\n')

Digital radiography requires significantly less radiation than conventional film-based radiography because the sensors used are more sensitive to x-rays. This increased sensitivity allows for shorter exposure times, typically reducing the required radiation dose by 70 to 80 percent compared to traditional film methods. As a result, patients receive a lower absorbed dose of radiation, which is a crucial advantage in minimizing potential health risks associated with x-ray exposure.


In [53]:
response = conversational_rag_chain.invoke(
        {"input": "how it is used?"},
        config={
            "configurable": {"session_id": "456"}
        },
    )["answer"]
print(response, end='\n')

Digital radiography is used by positioning a digital sensor or phosphor storage plate (PSP) in the patient's mouth, similar to traditional film placement. The conventional x-ray machine is then activated to expose the sensor to x-radiation, capturing the image as electronic signals. These signals are processed and displayed almost instantaneously on a computer monitor, allowing for immediate review and analysis by dental professionals.


In [62]:
store.keys()

dict_keys(['4565', '1234', '456'])

In [66]:
print(store['456'],end='\n')

Human: Advantages and Disadvantages of Digital Radiography 
AI: **Advantages:**
1. Immediate viewing of images, allowing for quicker clinical decisions.
2. Requires significantly less radiation compared to conventional film-based radiography.
3. Eliminates the need for chemical processing and allows for easy electronic transfer of images.

**Disadvantages:**
1. High initial setup costs for digital radiography systems.
2. Sensors can be fragile and expensive to replace if damaged.
3. Users may require time to learn and become comfortable with the software.
Human: Expand on point number 2
AI: Digital radiography requires significantly less radiation than conventional film-based radiography because the sensors used are more sensitive to x-rays. This increased sensitivity allows for shorter exposure times, typically reducing the required radiation dose by 70 to 80 percent compared to traditional film methods. As a result, patients receive a lower absorbed dose of radiation, which is a cruc